# [Reference](https://keras.io/examples/vision/mobilevit/)

# Introduction

In this example, we implement the MobileViT architecture (Mehta et al.), which combines the benefits of Transformers (Vaswani et al.) and convolutions. With Transformers, we can capture long-range dependencies that result in global representations. With convolutions, we can capture spatial relationships that model locality.

Besides combining the properties of Transformers and convolutions, the authors introduce MobileViT as a general-purpose mobile-friendly backbone for different image recognition tasks. Their findings suggest that, performance-wise, MobileViT is better than other models with the same or higher complexity (MobileNetV3, for example), while being efficient on mobile devices.

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#allow to work on hertie server
os.environ['TORCH_HOME']= '/workspace/cache'

#!pip3 install tensorflow-addons
#!pip3 install tensorflow-hub

#!pip3 install --upgrade typing-extensions
#!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from PIL import Image
import os, cv2, gc
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras, Module 
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import losses
from tensorflow.keras import optimizers
import tensorflow_hub as hub
from keras.applications import imagenet_utils
import tensorflow_datasets as tfds

#import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from keras.layers.advanced_activations import LeakyReLU

try:
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass 

# enable mixed_precision and jit compiler 
tf.keras.mixed_precision.set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)
#allow gpu to work 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0


In [6]:
INP_SIZE      = (512, 512) # Input size of the Image Resizer Module (IRM)
TARGET_SIZE   = (224, 224) # Output size of IRM and Input size of the Vision Transformer 
INTERPOLATION = "bilinear"
N_CLASSES = 15587
#config_head = 'arcface'


NUM_FOLDS  = 5
BATCH_SIZE = 24
SEED       = 42

DATA_DIR  = '../happy-whale/input/'
TRAIN_DIR = DATA_DIR + 'cropped_train_images/cropped_train_images/'
TEST_DIR  = DATA_DIR + 'cropped_test_images/cropped_test_images/'

# SetAutoTune
AUTOTUNE = tf.data.AUTOTUNE  

In [7]:
path = '../happy-whale/input/cropped_train_images/cropped_train_images/'

for file in os.listdir(path): 
    if not file.endswith(".jpg"): 
        img = Image.open("{path}/{file}")
        file_name, file_ext = os.path.splitext(file)
        img.save('/jpg/{}.jpg'.format(file_name))

# Prepare tf.data object

In [8]:
#https://www.kaggle.com/ipythonx/tf-keras-learning-to-resize-image-for-vit-model/notebook
def build_augmenter(is_labelled):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.65, 1.05)
        img = tf.image.random_brightness(img, 0.05)
        img = tf.image.random_contrast(img, 0.75, 1.05)
        img = tf.image.random_hue(img, 0.05)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    return augment_with_labels if is_labelled else augment

def build_decoder(is_labelled, size):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        img = tf.image.decode_jpeg(file_bytes, channels = 3)
        img = tf.image.resize(img, (size[0], size[1]))
        return tf.cast(tf.divide(img, 255.),tf.float32)
    
    def decode_with_labels(path, label):
        label = tf.cast(label, tf.int32)
        return decode(path),label
    
    return decode_with_labels if is_labelled else decode

def create_dataset(df, 
                   batch_size  = 32, 
                   is_labelled = False, 
                   augment     = False, 
                   repeat      = False, 
                   shuffle     = False,
                   size        = INP_SIZE):
    decode_fn    = build_decoder(is_labelled, size)
    augmenter_fn = build_augmenter(is_labelled)
    
    # Create Dataset
    if is_labelled:
        dataset = tf.data.Dataset.from_tensor_slices((df['Id'].values, df['target_value'].values))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df['Id'].values))
        
    dataset = dataset.map(decode_fn, num_parallel_calls = AUTOTUNE)
    dataset = dataset.map(augmenter_fn, num_parallel_calls = AUTOTUNE) if augment else dataset
    dataset = dataset.repeat() if repeat else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration = True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [9]:
label_encoder = LabelEncoder()


In [10]:
# Load Train Data
train_df = pd.read_csv(f'{DATA_DIR}train.csv')
train_df['Id'] = train_df['image'].apply(lambda x: f'{TRAIN_DIR}{x}')

# Adjust typos in "species" column from Andrada's kernel
train_df["species"] = train_df["species"].replace(["bottlenose_dolpin", "kiler_whale",
                                             "beluga", 
                                             "globis", "pilot_whale"],
                                            ["bottlenose_dolphin", "killer_whale",
                                             "beluga_whale", 
                                             "short_finned_pilot_whale", "short_finned_pilot_whale"])


# Set a specific label to be able to perform stratification
#train_df['stratify_label'] = train_df['individual_id']

train_df['target_value']  = label_encoder.fit_transform(train_df['individual_id'] )

# Summary
print(f'train_df: {train_df.shape}')
train_df.head()

train_df: (51033, 6)


,image,species,individual_id,box,Id,target_value
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,2 116 802 665,../happy-whale/input/cropped_train_images/crop...,12348
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,588 597 3504 1477,../happy-whale/input/cropped_train_images/crop...,1636
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,0 453 3183 1589,../happy-whale/input/cropped_train_images/crop...,5842
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,1 91 2636 1495,../happy-whale/input/cropped_train_images/crop...,4551
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,1139 1590 3574 1913,../happy-whale/input/cropped_train_images/crop...,8721


In [11]:
train_df[train_df['individual_id']=='19fbb960f07d']

,image,species,individual_id,box,Id,target_value
10,001001f099519f.jpg,minke_whale,19fbb960f07d,788 1036 2101 1373,../happy-whale/input/cropped_train_images/crop...,1613
255,0164d8fd7a7cfe.jpg,minke_whale,19fbb960f07d,676 640 2517 1144,../happy-whale/input/cropped_train_images/crop...,1613
345,01e775c518a824.jpg,minke_whale,19fbb960f07d,12 641 2512 1425,../happy-whale/input/cropped_train_images/crop...,1613
393,02278d7a7d207e.jpg,minke_whale,19fbb960f07d,1279 960 2630 1373,../happy-whale/input/cropped_train_images/crop...,1613
528,02d21d4df0b87f.jpg,minke_whale,19fbb960f07d,2347 1222 2536 1368,../happy-whale/input/cropped_train_images/crop...,1613
...,...,...,...,...,...,...
49109,f653df18636e4b.jpg,minke_whale,19fbb960f07d,1778 1042 2308 1215,../happy-whale/input/cropped_train_images/crop...,1613
49437,f8133d5fc02ba1.jpg,minke_whale,19fbb960f07d,473 1060 2162 1511,../happy-whale/input/cropped_train_images/crop...,1613
49879,fa2d984b021ec2.jpg,minke_whale,19fbb960f07d,1485 1080 2194 1252,../happy-whale/input/cropped_train_images/crop...,1613
50229,fbef59e7c8918c.jpg,minke_whale,19fbb960f07d,1225 925 2910 1166,../happy-whale/input/cropped_train_images/crop...,1613


In [12]:
train_df.species.value_counts()

bottlenose_dolphin           10781
beluga_whale                  7443
humpback_whale                7392
blue_whale                    4830
false_killer_whale            3326
dusky_dolphin                 3139
killer_whale                  2455
spinner_dolphin               1700
melon_headed_whale            1689
minke_whale                   1608
fin_whale                     1324
gray_whale                    1123
southern_right_whale           866
short_finned_pilot_whale       745
spotted_dolphin                490
sei_whale                      428
common_dolphin                 347
cuviers_beaked_whale           341
long_finned_pilot_whale        238
white_sided_dolphin            229
brydes_whale                   154
pantropic_spotted_dolphin      145
commersons_dolphin              90
pygmy_killer_whale              76
rough_toothed_dolphin           60
frasiers_dolphin                14
Name: species, dtype: int64

In [13]:
train_df.isnull().sum()

image                0
species              0
individual_id        0
box              15362
Id                   0
target_value         0
dtype: int64

In [14]:
train_ds, val_ds = train_df[:30000], train_df[30000:]

In [15]:
# Load Test Data
test_df = pd.read_csv(f'{DATA_DIR}test.csv')
test_df['Id'] = test_df['image'].apply(lambda x: f'{TEST_DIR}{x}')
test_df['individual_id'] = 0

# Summary
print(f'test_df: {test_df.shape}')
test_df.head()

test_df: (27956, 5)


,image,predictions,box,Id,individual_id
0,000110707af0ba.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,329 874 2934 1271,../happy-whale/input/cropped_test_images/cropp...,0
1,0006287ec424cb.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,1039 1636 1833 1875,../happy-whale/input/cropped_test_images/cropp...,0
2,000809ecb2ccad.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,787 787 1541 948,../happy-whale/input/cropped_test_images/cropp...,0
3,00098d1376dab2.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,77 228 2029 714,../happy-whale/input/cropped_test_images/cropp...,0
4,000b8d89c738bd.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,4 20 289 317,../happy-whale/input/cropped_test_images/cropp...,0


# MobileViT

MobileViT architecture comprised of following blocks
* Strided 3x3 convolutions that process the input image
* MobileNetV2-style inverted residual blocks for downsampling the resolution of the intermediate feature maps
* MobileViT blocks that combine the benefits of Transformers and convolutions. It is presented in the figure below (taken from the original paper):

![Screenshot from 2022-03-16 22-52-00.png](attachment:90baadc6-81f6-440f-8136-b0289236341f.png)

In [16]:
# Values are from table 4.
patch_size = 4  # 2x2, for the Transformer blocks.
image_size = 256
expansion_factor = 2  # expansion factor for the MobileNetV2 blocks.

In [17]:
def conv_block(x, filters=16, kernel_size=3, strides=2):
    conv_layer = layers.Conv2D(
        filters, kernel_size, strides=strides, activation=tf.nn.swish, padding="same"
    )
    return conv_layer(x)


# Reference: https://git.io/JKgtC


def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
    m = layers.Conv2D(expanded_channels, 1, padding="same", use_bias=False)(x)
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    if strides == 2:
        m = layers.ZeroPadding2D(padding=imagenet_utils.correct_pad(m, 3))(m)
    m = layers.DepthwiseConv2D(
        3, strides=strides, padding="same" if strides == 1 else "valid", use_bias=False
    )(m)
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    m = layers.Conv2D(output_channels, 1, padding="same", use_bias=False)(m)
    m = layers.BatchNormalization()(m)

    if tf.math.equal(x.shape[-1], output_channels) and strides == 1:
        return layers.Add()([m, x])
    return m


# Reference:
# https://keras.io/examples/vision/image_classification_with_vision_transformer/


def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.swish)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, x])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=[x.shape[-1] * 2, x.shape[-1]], dropout_rate=0.1,)
        # Skip connection 2.
        x = layers.Add()([x3, x2])

    return x


def mobilevit_block(x, num_blocks, projection_dim, strides=1):
    # Local projection with convolutions.
    local_features = conv_block(x, filters=projection_dim, strides=strides)
    local_features = conv_block(
        local_features, filters=projection_dim, kernel_size=1, strides=strides
    )

    # Unfold into patches and then pass through Transformers.
    num_patches = int((local_features.shape[1] * local_features.shape[2]) / patch_size)
    non_overlapping_patches = layers.Reshape((patch_size, num_patches, projection_dim))(
        local_features
    )
    global_features = transformer_block(
        non_overlapping_patches, num_blocks, projection_dim
    )

    # Fold into conv-like feature-maps.
    folded_feature_map = layers.Reshape((*local_features.shape[1:-1], projection_dim))(
        global_features
    )

    # Apply point-wise conv -> concatenate with the input features.
    folded_feature_map = conv_block(
        folded_feature_map, filters=x.shape[-1], kernel_size=1, strides=strides
    )
    local_global_features = layers.Concatenate(axis=-1)([x, folded_feature_map])

    # Fuse the local and global features using a convoluion layer.
    local_global_features = conv_block(
        local_global_features, filters=projection_dim, strides=strides
    )

    return local_global_features


# More on the MobileViT block:

*     First, the feature representations (A) go through convolution blocks that capture local relationships. The expected shape of a single entry here would be (h, w, num_channels).
*     Then they get unfolded into another vector with shape (p, n, num_channels), where p is the area of a small patch, and n is (h * w) / p. So, we end up with n non-overlapping patches.
*     This unfolded vector is then passed through a Tranformer block that captures global relationships between the patches.
*     The output vector (B) is again folded into a vector of shape (h, w, num_channels) resembling a feature map coming out of convolutions.

Vectors A and B are then passed through two more convolutional layers to fuse the local and global representations. Notice how the spatial resolution of the final vector remains unchanged at this point. The authors also present an explanation of how the MobileViT block resembles a convolution block of a CNN. For more details, please refer to the original paper.

Next, we combine these blocks together and implement the MobileViT architecture (XXS variant). The following figure (taken from the original paper) presents a schematic representation of the architecture:

![Screenshot from 2022-03-16 22-55-19.png](attachment:7e6962e8-de9f-425b-b4a8-4507d44827d1.png)

In [18]:
def create_mobilevit(num_classes=N_CLASSES):
    inputs = keras.Input((image_size, image_size, 3))
    
    #version 2.7 tf x = layers.Rescaling(scale=1.0 / 255)(inputs)
    x = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1.0 / 255)(inputs)

    # Initial conv-stem -> MV2 block.
    x = conv_block(x, filters=16)
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=16
    )

    # Downsampling with MV2 block.
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=24, strides=2
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )

    # First MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=48, strides=2
    )
    x = mobilevit_block(x, num_blocks=2, projection_dim=64)

    # Second MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=64 * expansion_factor, output_channels=64, strides=2
    )
    x = mobilevit_block(x, num_blocks=4, projection_dim=80)

    # Third MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=80 * expansion_factor, output_channels=80, strides=2
    )
    x = mobilevit_block(x, num_blocks=3, projection_dim=96)
    x = conv_block(x, filters=320, kernel_size=1, strides=1)

    # Classification head.
    x = layers.GlobalAvgPool2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes,kernel_regularizer='l2', activation="softmax")(x)

    return keras.Model(inputs, outputs)


mobilevit_xxs = create_mobilevit()
#mobilevit_xxs.summary()

In [19]:
mobilevit_xxs.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 256, 256, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  448         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

 add_1 (Add)                    (None, 64, 64, 24)   0           ['batch_normalization_8[0][0]',  
                                                                  'batch_normalization_5[0][0]']  
                                                                                                  
 conv2d_7 (Conv2D)              (None, 64, 64, 48)   1152        ['add_1[0][0]']                  
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 64, 64, 48)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 tf.nn.silu_6 (TFOpLambda)      (None, 64, 64, 48)   0           ['batch_normalization_9[0][0]']  
                                                                                                  
 depthwise

                                                                                                  
 add_5 (Add)                    (None, 4, 256, 64)   0           ['multi_head_attention_1[0][0]', 
                                                                  'add_4[0][0]']                  
                                                                                                  
 layer_normalization_3 (LayerNo  (None, 4, 256, 64)  128         ['add_5[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 dense_2 (Dense)                (None, 4, 256, 128)  8320        ['layer_normalization_3[0][0]']  
                                                                                                  
 dropout_2 (Dropout)            (None, 4, 256, 128)  0           ['dense_2[0][0]']                
          

 eadAttention)                                                    'layer_normalization_6[0][0]']  
                                                                                                  
 add_9 (Add)                    (None, 4, 64, 80)    0           ['multi_head_attention_3[0][0]', 
                                                                  'add_8[0][0]']                  
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 4, 64, 80)   160         ['add_9[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 dense_6 (Dense)                (None, 4, 64, 160)   12960       ['layer_normalization_7[0][0]']  
                                                                                                  
 dropout_6

                                                                                                  
 depthwise_conv2d_6 (DepthwiseC  (None, 8, 8, 160)   1440        ['zero_padding2d_3[0][0]']       
 onv2D)                                                                                           
                                                                                                  
 batch_normalization_19 (BatchN  (None, 8, 8, 160)   640         ['depthwise_conv2d_6[0][0]']     
 ormalization)                                                                                    
                                                                                                  
 tf.nn.silu_13 (TFOpLambda)     (None, 8, 8, 160)    0           ['batch_normalization_19[0][0]'] 
                                                                                                  
 conv2d_22 (Conv2D)             (None, 8, 8, 80)     12800       ['tf.nn.silu_13[0][0]']          
          

                                                                                                  
 dropout_17 (Dropout)           (None, 4, 16, 96)    0           ['dense_17[0][0]']               
                                                                                                  
 add_20 (Add)                   (None, 4, 16, 96)    0           ['dropout_17[0][0]',             
                                                                  'add_19[0][0]']                 
                                                                                                  
 reshape_5 (Reshape)            (None, 8, 8, 96)     0           ['add_20[0][0]']                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 8, 8, 80)     7760        ['reshape_5[0][0]']              
                                                                                                  
 concatena

In [20]:
batch_size = 128
auto = tf.data.AUTOTUNE
resize_bigger = 280

In [21]:
individual_id_group = train_df['target_value'].copy().to_list()

# Training , Valdation Dataset 

In [22]:
trn_ds, val_ds = train_test_split(train_df,test_size=0.33, random_state=42) 

In [23]:
#train_df =  train_df.sample(frac = 0.1)
trn_ds.shape, val_ds.shape

((34192, 6), (16841, 6))

In [24]:
training_dataset = create_dataset(trn_ds,
                                  batch_size  = BATCH_SIZE, 
                                  is_labelled = True, 
                                  augment = True,
                                  repeat  = False, 
                                  shuffle = False,
                                 size = (image_size, image_size))
#sample_train_images, _ = next(iter(training_dataset))

In [25]:
batch, labels = next(iter(training_dataset))

In [26]:
plt.figure(figsize=(16, 10))
for i, image in enumerate(batch[:20]):
    ax = plt.subplot(5, 4, i + 1)
    plt.title(labels[i].numpy())
    plt.imshow(image.numpy().squeeze())
    plt.axis("off")


plt.tight_layout()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [27]:
val_dataset = create_dataset(val_ds,
                                  batch_size  = BATCH_SIZE, 
                                  is_labelled = True, 
                                  augment = True,
                                  repeat  = False, 
                                  shuffle = False,
                            size = (image_size, image_size))
#sample_val_images, _ = next(iter(val_dataset))

In [28]:
batch, labels = next(iter(val_dataset))

In [29]:
plt.figure(figsize=(16, 10))
for i, image in enumerate(batch[:20]):
    ax = plt.subplot(5, 4,  i + 1)
    plt.title(labels[i].numpy())
    plt.imshow(image.numpy().squeeze())
    plt.axis("off")


plt.tight_layout()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


# Train a MobileViT (XXS) model

In [30]:
#unq_lbls = train_df.target_value.unique()

from sklearn.utils import class_weight
class_wgt = dict(zip(np.unique(train_df.target_value), class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(train_df.target_value), 
                y = train_df.target_value))) 
#class_wgt = class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(train_df.target_value),y = train_df.target_value )

In [31]:
learning_rate = 0.002
label_smoothing_factor = 0.1
epochs = 3

#optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate,decay=1e-6, momentum=0.9, clipnorm=1. )
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing_factor)
loss_fn = keras.losses.SparseCategoricalCrossentropy()


def run_experiment(model, fold, epochs=epochs):
    tf.keras.backend.clear_session()
    gc.collect()
    if fold < 1 :
        
        model.compile(optimizer=optimizer, 
                              loss=loss_fn,
                              #metrics = ['accuracy'])
                              metrics = [tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)])

        checkpoint_filepath = "/tmp/checkpoint"
        checkpoint_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_sparse_categorical_accuracy",
            save_best_only=True,
            save_weights_only=True,
        )


        trn = train_df.iloc[train_index]
        val = train_df.iloc[val_index]
        training_dataset = create_dataset(trn, 
                                          batch_size  = BATCH_SIZE, 
                                          is_labelled = True, 
                                          augment     = True, 
                                          repeat      = False, 
                                          shuffle     = False,
                                         size = (image_size, image_size))
        val_dataset = create_dataset(val, 
                                            batch_size  = BATCH_SIZE, 
                                            is_labelled = True,
                                            augment     = True, 
                                            repeat      = False,
                                            shuffle     = False,
                                           size = (image_size, image_size))


        history = model.fit(
            training_dataset,
            validation_data=val_dataset,
            epochs=epochs,
            class_weight = class_wgt,
            callbacks=[checkpoint_callback],
        )
        #model.summary()
        model.load_weights(checkpoint_filepath)
        _,_, accuracy = model.evaluate(val_dataset)
        print(f"Validation accuracy: {round(accuracy * 100, 2)}%")
    



**We would like to see the images related to individual ids, so taken the samples base on  GroupKFold here.**

*[Please Refer here for more inforamtion on GroupKfold](https://www.kaggle.com/reighns/groupkfold-and-stratified-groupkfold-efficientnet)*

In [32]:
from sklearn.model_selection import StratifiedKFold, GroupKFold

# OOF RMSE Placeholder

kfold = GroupKFold(n_splits = NUM_FOLDS)
for fold, (train_index, val_index) in enumerate(kfold.split(train_df.index, train_df['target_value'], groups=individual_id_group)):
    mobilevit_xxs = create_mobilevit(num_classes=N_CLASSES)    
    
    run_experiment(mobilevit_xxs, fold)    

Epoch 1/3
1702/1702 [==============================] - 235s 111ms/step - loss: nan - sparse_categorical_accuracy: 4.1640e-04 - sparse_top_k_categorical_accuracy: 0.0016 - val_loss: nan - val_sparse_categorical_accuracy: 0.0000e+00 - val_sparse_top_k_categorical_accuracy: 0.0000e+00
Epoch 2/3
1702/1702 [==============================] - 195s 115ms/step - loss: nan - sparse_categorical_accuracy: 2.4494e-05 - sparse_top_k_categorical_accuracy: 0.0000e+00 - val_loss: nan - val_sparse_categorical_accuracy: 0.0000e+00 - val_sparse_top_k_categorical_accuracy: 0.0000e+00
Epoch 3/3
426/426 [==============================] - 9s 21ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00 - sparse_top_k_categorical_accuracy: 0.0000e+00
Validation accuracy: 0.0%


In [33]:
'''
plt.figure(figsize=(8,8), tight_layout=True)
plt.subplot(211)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title(' Loss')
plt.ylabel('Error')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')

plt.subplot(212)
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('SparseCategoricalAccuracy Metric')
plt.ylabel('Error')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
'''

"\nplt.figure(figsize=(8,8), tight_layout=True)\nplt.subplot(211)\nplt.plot(history.history['loss'])\nplt.plot(history.history['val_loss'])\nplt.title(' Loss')\nplt.ylabel('Error')\nplt.xlabel('Epoch')\nplt.legend(['train', 'val'], loc='upper right')\n\nplt.subplot(212)\nplt.plot(history.history['sparse_categorical_accuracy'])\nplt.plot(history.history['val_sparse_categorical_accuracy'])\nplt.title('SparseCategoricalAccuracy Metric')\nplt.ylabel('Error')\nplt.xlabel('Epoch')\nplt.legend(['train', 'val'], loc='upper left')\nplt.show()\n"

In [34]:
# Serialize the model as a SavedModel.
mobilevit_xxs.save("mobilevit_xxs")

INFO:tensorflow:Assets written to: mobilevit_xxs/assets


INFO:tensorflow:Assets written to: mobilevit_xxs/assets


# Inference

In [35]:
test_dataset = create_dataset(test_df,
                                  batch_size  = BATCH_SIZE, 
                                  is_labelled = False, 
                                  augment = False,
                                  repeat  = False, 
                                  shuffle = False,
                            size = (image_size, image_size))
test_images = next(iter(test_dataset))


In [36]:
plt.figure(figsize=(16, 10))
for i, image in enumerate(test_images[:20]):
    ax = plt.subplot(5, 4,  i + 1)
    #plt.title(" Image")
    plt.imshow(image.numpy().squeeze())
    plt.axis("off")

plt.tight_layout()


In [37]:
#batch = next(iter(test_dataset))


In [38]:
dict_ids =dict(zip( train_df['target_value'], train_df['individual_id']))

In [39]:
test_preds = mobilevit_xxs.predict(test_dataset)

In [40]:
#dict_ids.get(1104)

In [41]:
'''
for i in range(2):
    arr = test_preds[i].argsort()[-top_n:][::-1]
    print (arr)
    r  = [''.join(dict_ids.get(arr[j])) for j in range(top_n)]
    print (r)
'''

"\nfor i in range(2):\n    arr = test_preds[i].argsort()[-top_n:][::-1]\n    print (arr)\n    r  = [''.join(dict_ids.get(arr[j])) for j in range(top_n)]\n    print (r)\n"

In [42]:
top_n = 5
for i in range(len(test_preds)):
    arr = test_preds[i].argsort()[-top_n:][::-1]
    #print (arr)
    test_df['predictions'][i]  = [''.join(dict_ids.get(arr[j])) for j in range(top_n)]
    #print (r)
    #test_df['predictions'][i] = r


<ipython-input-42-a8d11cfb969f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions'][i]  = [''.join(dict_ids.get(arr[j])) for j in range(top_n)]


In [43]:
sub_df = test_df[['image', 'predictions']]

In [44]:
sub_df.to_csv('submission.csv', index = False)

In [45]:
!tail submission.csv

ffe1e7587409a4.jpg,"['6ec0697b3511', 'd7b71850df75', '63688765c3c5', '0d1d09bc7782', 'b29550f1dc3b']"
fff024257c4246.jpg,"['3a544f9150cd', '9267dd80fa4e', '0d1d09bc7782', '64e1844c65f9', 'b9502efd2ef2']"
fff0f68c5b4c4c.jpg,"['d7b71850df75', '6ec0697b3511', '63688765c3c5', '0d1d09bc7782', 'b29550f1dc3b']"
fff2466b650dd7.jpg,"['3a544f9150cd', '0d1d09bc7782', '9267dd80fa4e', 'a1faa057fbb4', '86c548ebb760']"
fff6153701bae2.jpg,"['6ec0697b3511', 'd7b71850df75', '0d1d09bc7782', '63688765c3c5', 'b29550f1dc3b']"
fff6ff1989b5cd.jpg,"['0d1d09bc7782', '6ec0697b3511', '63688765c3c5', '16c95e4c6d13', 'b29550f1dc3b']"
fff8fd932b42cb.jpg,"['9267dd80fa4e', '0d1d09bc7782', '3a544f9150cd', '63688765c3c5', 'a1faa057fbb4']"
fff96371332c16.jpg,"['3a544f9150cd', '9267dd80fa4e', '0d1d09bc7782', '64e1844c65f9', 'a1faa057fbb4']"
fffc1c4d3eabc7.jpg,"['6ec0697b3511', '0d1d09bc7782', 'd7b71850df75', '63688765c3c5', 'b29550f1dc3b']"
fffc50be10c175.jpg,"['0d1d09bc7782', '6ec0697b3511', '3a544f9150cd', '86c548ebb760

Provide your feedback to help me improve further. Thank you.